# Data parsing

This function loads, cleans and filters the Excel data for the probabilistic risk assessment in the [main notebook](./borneo-probabilistic-risk-assessment.ipynb)

In brief, the data parsing process:
- Loads the data from the Excel files
- Ensures the relevant column types are numeric
- Renames columns to be consistent across the datasets
- Filters the tox data to:
    - Exclude tropical data
    - Exclude chronic exposures
    - Exclude exposure durations >96 hours
    - Exclude rows without values
- Filters the measured environmental concentration data to:
    - Exclude rows where concentration < limit of detection

In [4]:
import pandas as pd

def parse_data(mec_filepath, tox_filepath):
    """
    Load, clean and filter the measured concentrations and tox data
    for use in the probabilistic risk assessment
    
    Params
    ------
    tox_filepath
        Filepath to the toxicity Excel file
    mec_filepath
        Filepath to the measured environment concentration (MEC) Excel file
        
    Returns
    -------
    (df_mec, df_tox)
        Two dicts, each containing a water and soil dataframe, for MECs and
        tox data
    """
    
    compartments = ['water', 'soil']

    # Load the data and store the water and soil sheets in separate dfs
    df_tox = {
        'water': pd.read_excel(tox_filepath, sheet_name='LC50 freshwater'),
        'soil': pd.read_excel(tox_filepath, sheet_name='LC50 terrestrial')
    }
    df_mec = {
        'water': pd.read_excel(mec_filepath, sheet_name='Water - all'),
        'soil': pd.read_excel(mec_filepath, sheet_name='Soil - all')
    }
    
    # Remove whitespace from column, then convert to numeric
    df_mec['water']['conversion_to_universal'].astype(str).str.strip()
    df_mec['water']['conversion_to_universal'] = pd.to_numeric(
        df_mec['water']['conversion_to_universal'],
        errors='coerce'
    )
    df_tox['water']['Conc'] = df_tox['water']['Conc'].astype(float)
    df_tox['soil']['LC50 (mg/kg)'] = pd.to_numeric(
        df_tox['soil']['LC50 (mg/kg)'],
        errors='coerce'
    )
    
    # Remove whitespace from species column and rename columns to be consistent
    for c in compartments:
        df_tox[c]['Species'] = df_tox[c]['Species'].astype(str).str.strip()
        # Change any non-breaking spaces to normal spaces
        df_tox[c]['Species'] = df_tox[c]['Species'].astype(str).str.replace(u'\xa0', ' ')
        df_tox[c]['Species'] = df_tox[c]['Species'].astype(str).str.replace(u'\xa0', ' ')
        df_mec[c] = df_mec[c].rename(columns={'conversion_to_universal': 'MEC',
                                              'active_ingredient': 'chemical'})    
    df_tox['water'] = df_tox['water'].rename(columns={'Conc': 'LC50'})
    df_tox['soil'] = df_tox['soil'].rename(columns={'LC50 (mg/kg)': 'LC50'})
    
    # Empty dict to store our filtered data in
    df_tox_f = {}

    # Exclude rows where the climate column is 'Tropical'
    df_tox_f['water'] = df_tox['water'][df_tox['water']['climate'] != 'Tropical']
    # Only include rows where the exposure column is 'Acute'
    df_tox_f['water'] = df_tox_f['water'][df_tox_f['water']['exposure'] == 'Acute']
    
    # Change taxa names to be consistent
    df_tox_f['water'].loc[df_tox_f['water']['Taxa'].str.contains('Amphibia'), 'Taxa'] = 'Amphibian'
    df_tox_f['water'].loc[df_tox_f['water']['Taxa'].str.contains('Fish'), 'Taxa'] = 'Fish'
    df_tox_f['water'].loc[df_tox_f['water']['Taxa'].str.contains('Nematode'), 'Taxa'] = 'Nematoda'
    df_tox_f['water'].loc[df_tox_f['water']['Taxa'].str.contains('Annelid'), 'Taxa'] = 'Annelida'
    
    # Change chemical names to be consistent
    df_tox['soil'].loc[df_tox['soil']['Chemical'].str.contains('Glufosinate-ammonium'), 'Chemical'] = 'Glufosinate ammonium'
    df_tox_f['water'].loc[df_tox_f['water']['Chemical'].str.contains('Glufosinate-ammonium'), 'Chemical'] = 'Glufosinate ammonium'
    df_mec['soil'].loc[df_mec['soil']['chemical'].str.contains('metsulfuron-methyl'), 'chemical'] = 'Metsulfuron-methyl'

    # Exposure duration - the units are generally hours, but there
    # are a few measured in days, denoted by 'd' in the data.
    # We can use this as a way of filtering out those rows - 
    # by using the `to_numeric()` function, we can choose to ignore
    # rows that can't be converted to a numeric datatype (i.e.
    # those with 'd' in them). This also excludes rows that don't
    # have a value in the exposure column.
    df_tox_f['water'].loc[:,'Exposure Duration (h)'] = pd.to_numeric(
        df_tox_f['water']['Exposure Duration (h)'],
        errors='coerce'
    )
    # to_numeric() will put NaNs in the rows to exclude, and now
    # we need to actually filter them out
    df_tox_f['water'] = df_tox_f['water'].dropna(subset=['Exposure Duration (h)'])
    # Now all values should be in hours, so let's exclude those rows
    # with a value of >96
    df_tox_f['water'] = df_tox_f['water'][df_tox_f['water']['Exposure Duration (h)'] <= 96]

    # All we need to do for soils is remove NaNs
    df_tox_f['soil'] = df_tox['soil'].dropna(subset=['LC50'])

    # Finally, remove any rows where the concentration is zero (or <LOD)
    # so that we can log the data
    for c in compartments:
        df_tox_f[c] = df_tox_f[c][df_tox_f[c]['LC50'] > 0.0]
        df_mec[c] = df_mec[c][(df_mec[c]['MEC'] > 0.0) & (df_mec[c]['environmental concentration'] != 'ND')]
    
    return df_mec, df_tox_f